In [12]:
# 导入所需的库
import os
import pathlib
import pandas as pd
import numpy as np
import scipy
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.base import clone
from functools import partial


## 1.定义一些常量

In [13]:

DATA_DIR = pathlib.Path("D:/Amazon6")  # 数据文件所在的目录
SEED = 0  # 随机数种子
NUM_ITER = 10  # 迭代次数




## 2. 观察数据

In [14]:
# 读取训练集和测试集
train_df = pd.read_csv(DATA_DIR / "train.csv", sep='\t')
test_df = pd.read_csv(DATA_DIR / "test.csv", sep='\t')

# 查看数据集的大小和前几行
print(f"训练集大小：{len(train_df)}")
print(f"测试集大小：{len(test_df)}")
print("训练集前五行：")
print(train_df.head())
print("测试集前五行：")
print(test_df.head())

训练集大小：57039
测试集大小：11208
训练集前五行：
   reviewerID   asin                                         reviewText  \
0        7885   3901  First off, allow me to correct a common mistak...   
1       52087  47978  I am really troubled by this Story and Enterta...   
2        5701   3667  A near-perfect film version of a downright glo...   
3       47191  40892  Keep your expectations low.  Really really low...   
4       40957  15367  "they dont make em like this no more..."well.....   

   overall  votes_up  votes_all  label  
0      5.0         6          7      0  
1      3.0        99        134      0  
2      4.0        14         14      1  
3      1.0         4          7      0  
4      5.0         3          6      0  
测试集前五行：
   Id  reviewerID   asin                                         reviewText  \
0   0       82947  37386  I REALLY wanted this series but I am in SHOCK ...   
1   1       10154  23543  I have to say that this is a work of art for m...   
2   2        5789   5724  

## 3. 文本特征的处理，使用了TF-IDF（Term Frequency-Inverse Document Frequency）向量化方法。

TF-IDF 向量化：

TfidfVectorizer 是 scikit-learn 库中用于将文本转换为 TF-IDF 表示的工具。
stop_words='english' 意味着在向量化的过程中将会去除英语停用词，这些是在文本中常见但通常没有实际意义的单词。
word_model.fit_transform(train_df['reviewText']) 用于拟合并转换训练集的 'reviewText' 列，得到 TF-IDF 表示的训练集文本特征 train_X。
word_model.transform(test_df['reviewText']) 用于将测试集的 'reviewText' 列转换为 TF-IDF 表示，得到测试集文本特征 test_X。
拼接总评分特征：

将每个文本的总评分（'overall' 列）通过 scipy.sparse.hstack 水平拼接到对应的 TF-IDF 特征矩阵中。
这样，每个文本的 TF-IDF 特征矩阵将会包括其对应的总评分特征。总评分特征还被除以5，可能是为了归一化到0-1范围，因为总评分通常在1-5之间。
最终，train_X 和 test_X 包含了处理好的文本特征，可以用于训练和测试机器学习模型。这个过程是常见的在文本分类任务中，结合文本信息和其他特征以提高模型性能。

In [15]:
# tf/idf 处理文本特征
word_model = TfidfVectorizer(stop_words='english')
train_X = word_model.fit_transform(train_df['reviewText'])
test_X = word_model.transform(test_df['reviewText']) 

# 拼上总评分特征
train_X = scipy.sparse.hstack([train_X, train_df['overall'].values.reshape((-1, 1)) / 5])
test_X = scipy.sparse.hstack([test_X, test_df['overall'].values.reshape((-1, 1)) / 5])

## 4. Ensemble 算法实现
4.1这些函为了实现 bagging（自助采样集成）的一部分，其中通过构造多个分类器，每个分类器使用自助采样的样本进行训练，最后通过平均它们的预测概率来提高模型的鲁棒性。

In [16]:
# 定义一个函数，根据分类器名称构造分类器对象
def construct_clf(clf_name):
    clf = None
    if clf_name == 'SVM':
        clf = svm.LinearSVC()
    elif clf_name == 'DTree':
        clf = DecisionTreeClassifier(max_depth=10, class_weight='balanced')
    elif clf_name == 'NB':
        clf = BernoulliNB()
    clf = CalibratedClassifierCV(clf, cv=2, method='sigmoid')  # 概率校正
    return clf

# 定义一个函数，获取自助采样
def get_bootstrap_sample(X, Y):
    sample_idx = np.random.choice(len(Y), len(Y), replace=True)
    return X[sample_idx], Y[sample_idx]

# 定义一个函数，计算平均预测概率
def average_predictions(classifier, X, Y, num_iter, test_X):
    predictions = [classifier.fit(*get_bootstrap_sample(X, Y)).predict_proba(test_X)[:, 1]
                   for _ in range(num_iter)]
    return np.mean(predictions, axis=0)

4.2通过使用不同的自助采样训练数据，构造多个相似但略有不同的基本分类器，最后通过平均它们的预测结果来提高模型的性能和鲁棒性。 

In [17]:
# 定义一个类，实现Bagging算法
class Bagging:
    def __init__(self, clf, num_iter):
        self.clf = clf
        self.num_iter = num_iter
        
    def fit_predict(self, X, Y, test_X):
        clf = clone(self.clf)
        return average_predictions(clf, X, Y, self.num_iter, test_X)

4.3这些函数的组合用于 AdaBoost 算法的每一轮迭代，其中每个基本分类器的权重和性能都根据前一轮的表现进行调整。

In [18]:
# 定义一个函数，计算beta值
def compute_beta(error):
    return error / (1 - error)

# 定义一个函数，更新权重
def update_weights(weights, predictions, Y, beta):
    weights *= np.where(predictions == Y, beta, 1)
    return weights / weights.sum()

# 定义一个函数，计算加权预测概率
def weighted_predictions(classifier, X, Y, weights, test_X):
    classifier.fit(X, Y, sample_weight=weights)
    predictions = classifier.predict(X)
    predict_proba = classifier.predict_proba(test_X)[:, 1]
    return predictions, predict_proba

4.4通过构造一系列基本分类器，每个分类器的权重由前一轮的表现来调整，最终得到一个加权平均的预测结果。

In [21]:
# 定义一个类，实现AdaBoostM1算法
class AdaBoostM1:
    def __init__(self, clf, num_iter):
        self.clf = clf
        self.num_iter = num_iter
        
    def fit_predict(self, X, Y, test_X):
        num_samples = len(Y)
        clf = clone(self.clf)
        weights = np.ones(num_samples) / num_samples
        result_list = []
        beta_list = []

        for _ in range(self.num_iter):
            predictions, predict_proba = weighted_predictions(clf, X, Y, weights, test_X)
            error = np.dot(weights, predictions != Y)
            
            if error > 0.5:
                break

            beta = compute_beta(error)
            weights = update_weights(weights, predictions, Y, beta)
            beta_list.append(beta)
            result_list.append(predict_proba)

        beta_list = np.log(1 / np.array(beta_list))
        weights = beta_list / np.sum(beta_list)
        
        result = np.dot(weights, result_list)
        return result

## 5. 测试并生成结果

In [ ]:
# 设置随机数种子
np.random.seed(SEED)

# 构造分类器
CLF_NAME = 'SVM' # 分类器名称，可以是'DTree', 'SVM', 'NB'
clf = construct_clf(CLF_NAME)

# 选择算法
choose = Bagging(clf, NUM_ITER)
#choose = AdaBoostM1(clf, NUM_ITER)

# 训练并预测
y_predict = choose.fit_predict(train_X.tocsr(), train_df['label'], test_X.tocsr())

## 6. 生成提交结果

In [24]:
result_df = pd.DataFrame()
result_df['Id'] = test_df['Id'].values
result_df['Predicted'] = y_predict
result_df.to_csv("D:\RESULT.csv", index=False)